<a href="https://colab.research.google.com/github/Jomucon21muri/Aprendizaje_automatico/blob/main/01_Sistemas_aprendizaje_automatico/01_Ml_supervisado/ml_supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML supervisado

## Introducción

Bienvenido al notebook de **Machine learning supervisado**. Aquí implementaremos y compararemos los principales algoritmos de aprendizaje supervisado.

### Temas que veremos:

1. **Algoritmos de clasificación**
   - Regresión logística
   - Árboles de decisión
   - Random forest
   - Support vector machines (SVM)
   - K-nearest neighbors (KNN)

2. **Algoritmos de regresión**
   - Regresión lineal
   - Regresión polinomial
   - Ridge y Lasso
   - Gradient boosting regressor

3. **Preparación de datos**
   - Limpieza y normalización
   - División train/test
   - Feature engineering

4. **Evaluación de modelos**
   - Métricas de clasificación
   - Métricas de regresión
   - Comparación de algoritmos

---

**Librerías principales:**
- scikit-learn
- pandas
- numpy
- matplotlib/seaborn

**Duración estimada:** 8-10 horas

In [ ]:
# Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("Librerías importadas correctamente")
print("Listo para comenzar con ML supervisado")